In [40]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from uuid import uuid4
from pymongo import MongoClient
import urllib

In [41]:
df = pd.read_excel("../static/data/list_product.xlsx")
df = df[df["sub_cate_report"] != "TIKI-Ngon"]
df["product_id"] = df["product_id"].astype("int64")
df["sku"] = df["sku"].astype("int64")
df = df.head(1000)

In [42]:
df["url"] = df["product_id"].apply(lambda x: "https://tiki.vn/p/" + str(x))

In [43]:
df.head()

,product_id,sku,product_name,cate_report,sub_cate_report,nmv,url
0,187254763,3890696903664,Điện Thoại Samsung Galaxy A13 (4GB/64GB) - Hàn...,Phones - Tablets,Phones - Tablets,4.817524e+09,https://tiki.vn/p/187254763
1,187254761,1370851719923,Điện Thoại Samsung Galaxy A13 (4GB/64GB) - Hàn...,Phones - Tablets,Phones - Tablets,4.503317e+09,https://tiki.vn/p/187254761
2,176731589,8641272074110,Xe máy điện VinFast Feliz S - Trắng ngọc trai,Lifestyle,Automotive & Motorcycle,2.485170e+09,https://tiki.vn/p/176731589
3,76198708,9304471866341,Xe Motor điện VinFast Feliz (Đã bao gồm Sạc & ...,Lifestyle,Automotive & Motorcycle,2.308266e+09,https://tiki.vn/p/76198708
4,87105841,5245012177255,Phiếu mua Xe Máy Honda Thăng Long,Lifestyle,Automotive & Motorcycle,2.152700e+09,https://tiki.vn/p/87105841


In [32]:
url_eg = df.iloc[0,-1]
url_eg

'https://tiki.vn/p/187254763'

In [49]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url=url_eg)

/var/folders/4y/vhc9jlvn1v7b0h3qkld4mk0m0000gn/T/ipykernel_6060/261311818.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [47]:
uri = "mongodb+srv://vietthangc1:"+urllib.parse.quote_plus('f2bdx@*-uLAZz!f')+"@cluster0.le7ea.mongodb.net/test"
db = MongoClient(uri)["ecom"]

In [50]:
i = 0
for row in df.iterrows():
  item = row[1]
  i += 1

  brand = None
  price = None
  seller = None

  driver.get(url="https://tiki.vn/p/" + str(item["product_id"]))
  time.sleep(0.5)
  html = BeautifulSoup(driver.page_source)

  try:
    brand_div = html.find("div", class_ = "brand")
    brand = brand_div.find("span", class_ = "brand-and-author").find("a").get_text()
  except:
    pass

  try:
    price_div = html.find("div", class_ = "price-and-icon")

    flash_sale_div = price_div.find_all("div", class_ = "flash-sale-price")
    if len(flash_sale_div) > 0:
      price = flash_sale_div[0].find("span").get_text()
    else:
      price = price_div.find("div", class_ = "product-price__current-price").get_text()
    price = price.replace(".","").replace(" ₫", "")
    price = int(price)
  except:
    pass

  try: 
    img_div = html.find("div", class_ = "review-images")
    imgs = img_div.find_all("img")
    img_srcs = []
    for img in imgs:
      img_src = img.get("src")
      webs = requests.get(img_src)
      filename = uuid4().hex + "." + img_src.split(".")[-1]
      open("../static/img/product_img/" + filename, "wb").write(webs.content)
      img_srcs.append(filename)
  except:
    pass

  try:
    content_div = html.find_all("div", class_ = "content")[-1]
    lst_e = content_div.find("div").find("div").find_all()
    html_desc = ""
    for e in lst_e:
      html_desc += str(e)
  except:
    pass

  try:
    seller_div = html.find("div", class_ = "seller-info")
    seller = seller_div.find("span", class_ = "seller-name").find("span").get_text()
  except:
    pass

  dict = {
    "product_id": id,
    "product_name": item["product_name"],
    "cate_report": item["cate_report"],
    "sub_cate_report": item["sub_cate_report"],
    "brand": brand,
    "price": price,
    "seller": seller,
    "img_srcs": img_srcs,
    "description": html_desc
  }

  db.product.insert_one(dict)


KeyboardInterrupt: 

In [2]:
info = pd.DataFrame(dict)
info


NameError: name 'pd' is not defined

In [38]:
info["price"] = info["price"].str.replace(".","").str.replace(" ₫","")
info["price"] = info["price"].astype("int64")

/var/folders/4y/vhc9jlvn1v7b0h3qkld4mk0m0000gn/T/ipykernel_3830/1099871348.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  info["price"] = info["price"].str.replace(".","").str.replace(" ₫","")


In [39]:
merge = pd.merge(df, info, how="inner", on="product_id")
merge[["product_id", "sku", "product_name", "cate_report", "sub_cate_report", "brand", "price", "seller", "img_srcs", "description"]].to_csv("../static/data/product_data.csv", index=False)

In [31]:
for row in df.iterrows():
  print(row[1]["sku"])

3890696903664
1370851719923
8641272074110
9304471866341
5245012177255
8980713037420
2954306961948
8632767284726
2844060242967
1599223535607
7196209362779
5679549432792
9294750039362
4538173201400
6236034261134
9978880816411
2762543561269
7204262708114
1855065721860
8599677765592
9615466832730
4053787264987
2554576025120
7024225106561
8391602711438
3508831619592
6560155557782
1297769243041
6077265095269
8202119666584
7216637014775
5300970669528
2006178471561
1861827577323
9432199571799
2281460688726
6291656111972
5687918348294
9855082987184
1515085327266
8950391933952
8680267340921
3012380447870
8110773361891
4210352701633
9096259191706
1925587526987
2360127508376
5532405936751
6370877633411
9255921646436
5484781712744
9282998827055
3719222012639
1613885961542
8162320707376
8577673950587
6609200285648
3155273175451
5440086254136
2019614614446
3518213419675
4772788596860
8244152819694
5222298406982
1159701958028
9744505780660
4574894463154
3171557883402
2339810112534
9562491512415
609209